<a href="https://colab.research.google.com/github/levicmech/2022-knu-yolo/blob/Victor_Levitskymech/levicmech/task3/EmotionsClassifierNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import torch
from torchvision import transforms, datasets


train_nm = 'train'
val_nm = 'val'

data_transforms = {
    train_nm: transforms.Compose([
        transforms.Resize(128),
        transforms.RandomCrop(128),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    val_nm: transforms.Compose([
        transforms.Resize(128),
        transforms.CenterCrop(128),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


data_dir = '/content/drive/MyDrive/emotions_ds/emotions_ds'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in [train_nm, val_nm]}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=256,
                                             shuffle=True,) for x in [train_nm, val_nm]}

dataset_sizes = {x: len(image_datasets[x]) for x in [train_nm, val_nm]}
class_names = image_datasets[train_nm].classes

In [ ]:
class_names


In [ ]:
dataset_sizes

In [ ]:
image, label = next(iter(dataloaders[train_nm]))
image.numpy().shape


In [ ]:
label


In [ ]:
image = image.view(-1, *image.size()[2:])


In [ ]:
plt.figure(figsize=(8, 8))
ax = plt.subplot(1, 1, 1)
plt.imshow(image[0], cmap='binary_r')
plt.title(class_names[label[0]])
plt.axis('off')

In [ ]:
import torchvision
import torchvision.models as models
from torch import nn

model = torchvision.models.resnet152(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

n_inputs = model.fc.in_features
n_classes = len(class_names)

model.fc = nn.Sequential(nn.Flatten(),
                         nn.Linear(n_inputs, 128),
                         nn.ReLU(),
                         nn.Linear(128, n_classes),
                         nn.Softmax(dim=1))

model.aux_logits = False

In [ ]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

if torch.cuda.is_available():
    model.cuda()

def train_model(model, dataloaders, criterion, optimizer, epochs=15):
    for epoch in range(epochs):
        print(f'\n{epoch+1} / {epochs}')
        print('-'*20)
        
        for phase in [train_nm, val_nm]:
            if phase == train_nm:
                model.train()
            else:
                model.eval()
                
            running_loss = 0.0
            correct = 0
            
            for X, y in dataloaders[phase]:
                X, y = X.cuda(), y.cuda()
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == train_nm):
                    out = model(X)
                    loss = criterion(out, y)
                    _, preds = torch.max(out, 1)
                    
                    if phase == train_nm:
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * X.size(0)
                correct += torch.sum(preds == y.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = correct.double() / len(dataloaders[phase].dataset
                                               
            print(f'{phase}: loss {epoch_loss:.4f}, acc {epoch_acc:.3f}')

           
train_model(model, dataloaders, criterion, optimizer, 5)

In [ ]:
true_labels = torch.tensor([]).to(device)
labels = torch.tensor([]).to(device)

for batch, (X, y) in enumerate(dataloaders[val_nm]):
    X, y = X.to(device), y.to(device)
    true_labels = torch.cat((true_labels, y), 0)
    preds = model(X)
    _, preds = torch.max(preds, dim=1)
    labels = torch.cat((labels, preds), 0)
    

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print('Classification report \n')
print(classification_report(true_labels.cpu(), labels.cpu(), target_names=class_names))
conf_matrix = confusion_matrix(true_labels.cpu(), labels.cpu(), normalize='true')
print('Confusion matrix \n')
print(conf_matrix.round(2))


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, auc
fig = plt.figure(figsize=(10, 10))

n_classes = len(class_names)
lab_plt = [(true_labels.cpu() == float(i)) for i in range(n_classes)]
outs_plt = [(labels.cpu() == float(i)) for i in range(n_classes)]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(lab_plt[i], outs_plt[i])
    roc_auc[i] = auc(fpr[i], tpr[i])


plt.title('ROC curves', fontweight='bold', fontsize=10)
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label = f'class {class_names[i]}, auc = {roc_auc[i]:.3f}' )

plt.legend(loc = 'best')
plt.plot([0,1], [0,1], linestyle='--', color='yellow')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.show()


In [2]:
count = 0
for img, label in dataloaders[val_nm]:
    plt.figure(figsize=(16, 16))
    count += 1
    img = img.to(device)
    pred = model(img)
    pred = torch.max(pred, dim=1)
    img = img.view(-1, *img.size()[2:])
    for i in range(3):
        ax = plt.subplot(1, 3, 1+i)
        plt.imshow(img[i*3].cpu(), cmap='binary_r')
        plt.title(class_names[label[0+i]])
        plt.axis('off')
    if count == 3:
        break

NameError: ignored